# Setting up the environment. PLEASE WAIT 🙃

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!pip install rembg[gpu]

In [ ]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

In [ ]:
# Install library
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [3]:
ver_openpose = "v1.7.0"
! echo $ver_openpose

v1.7.0


In [ ]:
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

In [ ]:
# manually downloading openpose models
%%bash
gdown 1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh
unzip models.zip
mv /content/models/face/pose_iter_116000.caffemodel /content/openpose/models/face/pose_iter_116000.caffemodel
mv /content/models/hand/pose_iter_102000.caffemodel /content/openpose/models/hand/pose_iter_102000.caffemodel
mv /content/models/pose/body_25/pose_iter_584000.caffemodel /content/openpose/models/pose/body_25/pose_iter_584000.caffemodel
mv /content/models/pose/coco/pose_iter_440000.caffemodel /content/openpose/models/pose/coco/pose_iter_440000.caffemodel
mv /content/models/pose/mpi/pose_iter_160000.caffemodel /content/openpose/models/pose/mpi/pose_iter_160000.caffemodel
rm -rf models
rm models.zip

In [6]:
! cd openpose && mkdir build && cd build

In [ ]:
! cd openpose/build && cmake -DUSE_CUDNN=OFF -DBUILD_PYTHON=ON ..

In [ ]:
# ! cd openpose/build && cmake ..

In [ ]:
! cd openpose/build && make -j`nproc`
! cd openpose && mkdir output

In [9]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken <your_token>

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=b8ae0d70bccacfc72462262492c96e88942fd5e8113a0a9c6745caea83aad689
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
import os
%cd /content/
!rm -rf clothes-virtual-try-on
!git clone https://github.com/practice404/clothes-virtual-try-on.git
os.makedirs("/content/clothes-virtual-try-on/checkpoints")

/content
Cloning into 'clothes-virtual-try-on'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 154 (delta 16), reused 12 (delta 12), pack-reused 132
Receiving objects: 100% (154/154), 20.47 MiB | 33.87 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [11]:
!gdown --id 18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6 --output /content/clothes-virtual-try-on/checkpoints/alias_final.pth
!gdown --id 1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz --output /content/clothes-virtual-try-on/checkpoints/gmm_final.pth
!gdown --id 1d7lZNLh51Qt5Mi1lXqyi6Asb2ncLrEdC --output /content/clothes-virtual-try-on/checkpoints/seg_final.pth

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6
From (redirected): https://drive.google.com/uc?id=18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6&confirm=t&uuid=6db7053d-df6d-41e3-ac5b-2f924303335f
To: /content/clothes-virtual-try-on/checkpoints/alias_final.pth
100% 402M/402M [00:01<00:00, 254MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz
To: /content/clothes-virtual-try-on/checkpoints/gmm_final.pth
100% 76.2M/76.2M [00:00<00:00, 223MB/s]
/usr/local/lib/python3.10/dist-packages/gdown

In [12]:
!gdown --id 1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx --output /content/clothes-virtual-try-on/cloth_segm_u2net_latest.pth --no-cookies

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx
From (redirected): https://drive.google.com/uc?id=1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx&confirm=t&uuid=50dc2d49-15b3-47ed-905f-fc2455dfea07
To: /content/clothes-virtual-try-on/cloth_segm_u2net_latest.pth
100% 177M/177M [00:00<00:00, 178MB/s]


In [13]:
%cd /content/
!pip install ninja

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.7 MB/s eta 0:00:00


In [14]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints

Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (719/719), done.
remote: Compressing objects: 100% (568/568), done.
remote: Total 719 (delta 149), reused 611 (delta 140), pack-reused 0
Receiving objects: 100% (719/719), 3.88 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (149/149), done.
/content/Self-Correction-Human-Parsing


In [15]:
# downloading LIP dataset model
!gdown --id 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
!mv /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth /content/Self-Correction-Human-Parsing/checkpoints/final.pth

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=83091795-9ef5-449e-8d11-008bbe2238eb
To: /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth
100% 267M/267M [00:01<00:00, 210MB/s]


In [16]:
%cd /content/

/content


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [18]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:
!conda install --channel conda-forge featuretools --yes

In [ ]:
!pip install opencv-python torchgeometry

In [ ]:
!pip install torchvision

# Welcome to Virtual-Cloth-Assistant

> It'll take some extra time in first execution for setting up and downloading of model weights

In [22]:
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse openpose-img openpose-json")

In [28]:
from flask import Flask, request, send_file, jsonify
from flask_ngrok import run_with_ngrok
from PIL import Image
import base64
import io

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return jsonify("hello world");

@app.route("/api/transform", methods=['POST'])
def begin():
  make_dir()
  print("data recieved")
  cloth = request.files['cloth']
  model = request.files['model']

  cloth = Image.open(cloth.stream)
  model = Image.open(model.stream)

  cloth.save("/content/inputs/test/cloth/cloth.jpg")
  model.save("/content/inputs/test/image/model.jpg")

  # running script to compute the predictions
  os.system("python /content/clothes-virtual-try-on/run.py")

  # loading output
  op = os.listdir("/content/output")[0]
  op = Image.open(f"/content/output/{op}")
  buffer = io.BytesIO()
  op.save(buffer, 'png')
  buffer.seek(0)
  os.system("rm -rf /content/output/")
  return send_file(buffer, mimetype='image/gif')

In [29]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e793-34-123-73-186.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
data recieved


INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 14:36:09] "POST /api/transform HTTP/1.1" 200 -
